In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
temp2022 = pd.read_csv(r"F:\work\rudra sir\my_work\some_results\temperature_2022.csv")
temp2100 = pd.read_csv(r"F:\work\rudra sir\my_work\some_results\temperature_2100.csv")

In [6]:
# Convert temp2022 to have a 'Date' column in the same format as temp2100
temp2022['Date'] = temp2022['Month'].astype(str) + '/' + temp2022['Day'].astype(str) + '/' + temp2022['Year'].astype(str)

# Merge the two DataFrames on 'Date'
khpI_Temp = pd.merge(
    temp2022[['Date', 'Tavg']],
    temp2100[['Date', 'Forecast Temperature']],
    on='Date',
    how='outer'
)

           Date  Tavg  Forecast Temperature
0      1/1/1982  5.65                   NaN
1      1/1/1983  2.60                   NaN
2      1/1/1985  6.60                   NaN
3      1/1/1986  6.90                   NaN
4      1/1/1987  5.50                   NaN
...         ...   ...                   ...
42724  9/9/2096   NaN              15.17845
42725  9/9/2097   NaN              17.57140
42726  9/9/2098   NaN              16.86360
42727  9/9/2099   NaN              16.72425
42728  9/9/2100   NaN              18.21895

[42729 rows x 3 columns]


In [11]:
khpI_Temp['Date'] = pd.to_datetime(khpI_Temp['Date'], format='%m/%d/%Y')
khpI_Temp['Year'] = khpI_Temp['Date'].dt.year
khpI_Temp['Month'] = khpI_Temp['Date'].dt.month

In [13]:
# Step 3: Define baseline period (first 30 years of Tavg: 1982–2011)
baseline_data = khpI_Temp[(khpI_Temp['Year'] >= 1982) & (khpI_Temp['Year'] <= 2011)]['Tavg']
baseline_mean = baseline_data.mean()

# Step 4: Define future periods and compute mean for each
periods = {
    '2020s': (2011, 2040),
    '2050s': (2041, 2070),
    '2080s': (2071, 2100)
}


In [8]:
# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
# Step 2: Convert 'Date' to datetime and extract year and month
khpI_Temp['Date'] = pd.to_datetime(khpI_Temp['Date'], format='%m/%d/%Y', errors='coerce')
khpI_Temp['Year'] = khpI_Temp['Date'].dt.year
khpI_Temp['Month'] = khpI_Temp['Date'].dt.month

# Step 3: Define baseline period (1982–2011) and compute monthly means
baseline_data = khpI_Temp[(khpI_Temp['Year'] >= 1982) & (khpI_Temp['Year'] <= 2011)]
baseline_monthly_means = baseline_data.groupby('Month')['Tavg'].mean()
baseline_annual_mean = baseline_data['Tavg'].mean()

# Step 4: Define future periods
periods = {
    '2020s': (2011, 2040),
    '2050s': (2041, 2070),
    '2080s': (2071, 2100)
}

# Step 5: Calculate relative changes for each month and period
months = list(range(1, 13))  # Convert range to list
month_names = [pd.to_datetime(f'2025-{m}-01').strftime('%b') for m in months] + ['Annual']
relative_changes = {period: [] for period in periods.keys()}

for month in months + [0]:  # Now compatible: list + list
    if month == 0:
        baseline_mean = baseline_annual_mean
        # For annual, skip month filter and use only year range
        month_condition = slice(None)
    else:
        baseline_mean = baseline_monthly_means.get(month, float('nan'))
        month_condition = (khpI_Temp['Month'] == month)

    if pd.isna(baseline_mean):
        print(f"Warning: No baseline data for {'Annual' if month == 0 else month_names[month - 1]}. Using 0 for relative change.")
        for period in periods:
            relative_changes[period].append(0)
        continue

    for period_name, (start_year, end_year) in periods.items():
        # Apply year filter and month condition only if not annual
        if month == 0:
            period_data = khpI_Temp[(khpI_Temp['Year'] >= start_year) & 
                                  (khpI_Temp['Year'] <= end_year)]['Forecast Temperature']
        else:
            period_data = khpI_Temp[(khpI_Temp['Year'] >= start_year) & 
                                  (khpI_Temp['Year'] <= end_year) & 
                                  month_condition]['Forecast Temperature']
        period_mean = period_data.mean()
        if pd.isna(period_mean):
            print(f"Warning: No valid Forecast Temperature data for {'Annual' if month == 0 else month_names[month - 1]} in {period_name}.")
            relative_changes[period_name].append(0)
        else:
            relative_change = ((period_mean - baseline_mean) / baseline_mean) * 100
            relative_changes[period_name].append(relative_change)

# Step 6: Create DataFrame and save to CSV
baseline_values = [baseline_monthly_means.get(m, baseline_annual_mean) for m in range(1, 13)] + [baseline_annual_mean]
data_dict = {'Month': month_names, 'Baseline_Temperature': baseline_values}
for period in ['2020s', '2050s', '2080s']:
    data_dict[period] = relative_changes[period]
df = pd.DataFrame(data_dict)
df.to_csv('relative_change_temperature_data.csv', index=False)
print("Plotted data saved as 'relative_change_temperature_data.csv'.")

# Step 7: Create the dual-axis bar and line chart
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar charts for relative changes
bar_width = 0.25
x = range(len(month_names))
ax1.bar([i - bar_width for i in x], relative_changes['2020s'], bar_width, label='2020s', color='black', edgecolor='black')
ax1.bar(x, relative_changes['2050s'], bar_width, label='2050s', color='skyblue', edgecolor='black')
ax1.bar([i + bar_width for i in x], relative_changes['2080s'], bar_width, label='2080s', color='white', edgecolor='black')

# Set labels and title for left y-axis (relative change)
ax1.set_xlabel('Month')
ax1.set_ylabel('Relative Change in Temperature (%)', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.set_xticks(x)
ax1.set_xticklabels(month_names)
ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

# Right y-axis for baseline temperature
ax2 = ax1.twinx()
baseline_values = [baseline_monthly_means.get(m, baseline_annual_mean) for m in range(1, 13)] + [baseline_annual_mean]
ax2.plot(x, baseline_values, color='green', marker='o', label='Baseline temperature', linewidth=2)
ax2.set_ylabel('Baseline Temperature (°C)', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Title and legend
plt.title('(a) Relative Change in Forecasted Temperature (RCP 4.5, Baseline: 1982–2011)')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4)

# Adjust layout and save
plt.tight_layout()
plt.savefig('relative_change_temperature_dual_axis.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'relative_change_temperature_dual_axis.png'.")


Plotted data saved as 'relative_change_temperature_data.csv'.
Chart saved as 'relative_change_temperature_dual_axis.png'.


In [33]:
# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12



# Step 2: Convert 'Date' to datetime and extract year
khpI_Temp['Date'] = pd.to_datetime(khpI_Temp['Date'], format='%m/%d/%Y', errors='coerce')
khpI_Temp['Year'] = khpI_Temp['Date'].dt.year

# Step 3: Compute annual averages
# Group by Year and calculate mean for Tavg and Forecast Temperature
annual_observed = khpI_Temp.dropna(subset=['Tavg']).groupby('Year')['Tavg'].mean().reset_index()
annual_forecast = khpI_Temp.dropna(subset=['Forecast Temperature']).groupby('Year')['Forecast Temperature'].mean().reset_index()

# Step 4: Perform linear regression and calculate R²
def linear_regression_and_r2(x, y):
    # Fit linear trend line
    coeffs = np.polyfit(x, y, 1)  # Degree 1 for linear
    trend_line = np.poly1d(coeffs)
    # Calculate R²
    y_mean = np.mean(y)
    ss_tot = np.sum((y - y_mean) ** 2)
    ss_res = np.sum((y - trend_line(x)) ** 2)
    r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
    return coeffs, r_squared, trend_line

# For observed Tavg
if not annual_observed.empty:
    coeffs_obs, r2_obs, trend_obs = linear_regression_and_r2(annual_observed['Year'], annual_observed['Tavg'])
    equation_obs = f'y = {coeffs_obs[0]:.4f}x + {coeffs_obs[1]:.4f}\nR² = {r2_obs:.4f}'
else:
    equation_obs = "No data for Tavg"
    trend_obs = lambda x: np.full_like(x, np.nan)

# For Forecast Temperature
if not annual_forecast.empty:
    coeffs_fc, r2_fc, trend_fc = linear_regression_and_r2(annual_forecast['Year'], annual_forecast['Forecast Temperature'])
    equation_fc = f'y = {coeffs_fc[0]:.4f}x + {coeffs_fc[1]:.4f}\nR² = {r2_fc:.4f}'
else:
    equation_fc = "No data for Forecast Temperature"
    trend_fc = lambda x: np.full_like(x, np.nan)

# Step 5: Create DataFrame and save to CSV
if not annual_observed.empty or not annual_forecast.empty:
    # Combine data into a single DataFrame
    combined_data = pd.DataFrame()
    if not annual_observed.empty:
        combined_data = pd.concat([combined_data, annual_observed.rename(columns={'Tavg': 'Observed_Tavg'})], axis=1)
    if not annual_forecast.empty:
        combined_data = pd.concat([combined_data, annual_forecast.rename(columns={'Forecast Temperature': 'Forecast_Temperature'})], axis=1)
    # Drop duplicate 'Year' column if it exists
    combined_data = combined_data.loc[:, ~combined_data.columns.duplicated()]
    combined_data.to_csv('annual_temperature_data.csv', index=False)
    print("Plotted data saved as 'annual_temperature_data.csv'.")
else:
    print("No data available to save to CSV.")

# Step 6: Create the plot
plt.figure(figsize=(10, 6))

# Plot observed annual averages with trend line
if not annual_observed.empty:
    plt.plot(annual_observed['Year'], annual_observed['Tavg'], color='blue', label='Observed Tavg', linewidth=2)
    plt.plot(annual_observed['Year'], trend_obs(annual_observed['Year']), color='blue', linestyle='--', label='_nolegend_')

# Plot forecast annual averages with trend line
if not annual_forecast.empty:
    plt.plot(annual_forecast['Year'], annual_forecast['Forecast Temperature'], color='red', label='Forecast Temperature', linewidth=2)
    plt.plot(annual_forecast['Year'], trend_fc(annual_forecast['Year']), color='red', linestyle='--', label='_nolegend_')

# Annotations for equations and R²
if not annual_observed.empty:
    plt.text(0.02, 0.98, equation_obs, transform=plt.gca().transAxes, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
if not annual_forecast.empty:
    plt.text(0.02, 0.90, equation_fc, transform=plt.gca().transAxes, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
plt.title('Annual Average Observed Tavg and Forecast Temperature with Trend Lines')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Set y-axis range from 10 to 20°C
plt.ylim(10, 20)

# Adjust layout and save
plt.tight_layout()
plt.savefig('annual_temperature_trend_lines.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'annual_temperature_trend_lines.png'.")

Plotted data saved as 'annual_temperature_data.csv'.
Chart saved as 'annual_temperature_trend_lines.png'.


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12


# Step 2: Convert 'Date' to datetime and extract year and month
khpI_Temp['Date'] = pd.to_datetime(khpI_Temp['Date'], format='%m/%d/%Y')
khpI_Temp['Year'] = khpI_Temp['Date'].dt.year
khpI_Temp['Month'] = khpI_Temp['Date'].dt.month

# Step 3: Define baseline period (1982–2011) and compute monthly means
baseline_data = khpI_Temp[(khpI_Temp['Year'] >= 1982) & (khpI_Temp['Year'] <= 2011)]
baseline_monthly_means = baseline_data.groupby('Month')['Tavg'].mean()
baseline_annual_mean = baseline_data['Tavg'].mean()

# Step 4: Define future periods
periods = {
    '2020s': (2011, 2040),
    '2050s': (2041, 2070),
    '2080s': (2071, 2100)
}

# Step 5: Calculate average temperatures for each period
months = list(range(1, 13))
month_names = [pd.to_datetime(f'2025-{m}-01').strftime('%b') for m in months] + ['Annual']
period_means = {period: [] for period in ['Baseline'] + list(periods.keys())}

for month in months + [0]:
    if month == 0:
        baseline_mean = baseline_annual_mean
        month_condition = slice(None)
    else:
        baseline_mean = baseline_monthly_means.get(month, float('nan'))
        month_condition = (khpI_Temp['Month'] == month)

    # Baseline mean
    if month == 0:
        period_means['Baseline'].append(baseline_annual_mean)
    else:
        period_means['Baseline'].append(baseline_mean)

    # Future period means
    for period_name, (start_year, end_year) in periods.items():
        if month == 0:
            period_data = khpI_Temp[(khpI_Temp['Year'] >= start_year) & 
                                  (khpI_Temp['Year'] <= end_year)]['Forecast Temperature']
        else:
            period_data = khpI_Temp[(khpI_Temp['Year'] >= start_year) & 
                                  (khpI_Temp['Year'] <= end_year) & 
                                  month_condition]['Forecast Temperature']
        period_mean = period_data.mean()
        if pd.isna(period_mean):
            print(f"Warning: No valid Forecast Temperature data for {'Annual' if month == 0 else month_names[month - 1]} in {period_name}.")
            period_means[period_name].append(float('nan'))
        else:
            period_means[period_name].append(period_mean)

# Step 6: Create DataFrame and save to CSV
data_dict = {'Month': month_names}
for period in ['Baseline', '2020s', '2050s', '2080s']:
    data_dict[period] = period_means[period]
df = pd.DataFrame(data_dict)
df.to_csv('average_temperature_by_period_data.csv', index=False)
print("Plotted data saved as 'average_temperature_by_period_data.csv'.")

# Step 6: Create the bar chart
fig, ax = plt.subplots(figsize=(12, 6))

bar_width = 0.2
x = np.arange(len(month_names))

for i, period in enumerate(['Baseline', '2020s', '2050s', '2080s']):
    plt.bar(x + i * bar_width, period_means[period], bar_width, label=period, alpha=0.8)

plt.xlabel('Month')
plt.ylabel('Temperature (°C)')
plt.title('Average Temperature by Period (Baseline: 1982–2011, RCP 4.5)')
plt.xticks(x + bar_width * 1.5, month_names, rotation=45)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Set y-axis range from 10 to 30°C
plt.ylim(10, 30)

# Adjust layout and save
plt.tight_layout()
plt.savefig('average_temperature_by_period.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'average_temperature_by_period.png'.")

Plotted data saved as 'average_temperature_by_period_data.csv'.
Chart saved as 'average_temperature_by_period.png'.
